In [45]:
# imports and stuff
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns

The Data I chose: Spotify Music Dataset from kaggle ( https://www.kaggle.com/datasets/solomonameh/spotify-music-dataset )
It consits of two csv files; high popularity and low popularity

In [46]:
# Importing the data
high = pd.read_csv('../data/high_popularity_spotify_data.csv')
low = pd.read_csv('../data/low_popularity_spotify_data.csv')
# Could I create a merged dataset with an extra 'popularity' column to distinguish the two?

In [47]:
print('high', high.columns)
# print('high', low.columns)

print('high', high.shape)
print('low', high.shape)

high Index(['energy', 'tempo', 'danceability', 'playlist_genre', 'loudness',
       'liveness', 'valence', 'track_artist', 'time_signature', 'speechiness',
       'track_popularity', 'track_href', 'uri', 'track_album_name',
       'playlist_name', 'analysis_url', 'track_id', 'track_name',
       'track_album_release_date', 'instrumentalness', 'track_album_id',
       'mode', 'key', 'duration_ms', 'acousticness', 'id', 'playlist_subgenre',
       'type', 'playlist_id'],
      dtype='object')
high (1686, 29)
low (1686, 29)


Both have the same columns (and even the same amount of samples)
As they already have a 'track_popularity' column, I might not actually need to create one to distinguish the two,
if I know the threshold for high & low priority.\
Of course, the question arises, how chose this threshold and why. Maybe looking into the domain of this column can tell me more.\
I think a first interesting research question could be:"What parameters influence (the most), whether a song is popular or not

In [48]:
print('high max popularity level', high['track_popularity'].max())
print('high min popularity level', high['track_popularity'].min())
print('low max popularity level', low['track_popularity'].max())
print('low min popularity level' , low['track_popularity'].min())

high max popularity level 100
high min popularity level 68
low max popularity level 68
low min popularity level 11


Looks like the domain is 0-100, i.e. a percentage value and the threshold for low/high popularity seems to be 68.
Maybe, as both datasets have the same amount of observations, it's just the median of the whole dataset\
As songs with a popularity level of 68 are in both datasets, I'm assuming they just split in half\
Let's look at the 5 most popular and unpopular songs of the datasets

In [49]:
high = high.sort_values(by='track_popularity', ascending=False)
print("Most popular:\n", high[["track_name", "track_id", "track_artist", "track_album_name", "track_popularity", "playlist_name"]].head(5))

Most popular:
            track_name                track_id           track_artist  \
676  Die With A Smile  2plbrEY59IikOBgBGLjaoe  Lady Gaga, Bruno Mars   
455  Die With A Smile  2plbrEY59IikOBgBGLjaoe  Lady Gaga, Bruno Mars   
0    Die With A Smile  2plbrEY59IikOBgBGLjaoe  Lady Gaga, Bruno Mars   
677              APT.  5vNRhkKd0yEAg8suGBpjeY       ROSÉ, Bruno Mars   
4                APT.  5vNRhkKd0yEAg8suGBpjeY       ROSÉ, Bruno Mars   

     track_album_name  track_popularity      playlist_name  
676  Die With A Smile               100      Global Top 50  
455  Die With A Smile               100  Top Gaming Tracks  
0    Die With A Smile               100   Today's Top Hits  
677              APT.                98      Global Top 50  
4                APT.                98   Today's Top Hits  


No Taylor Swift, what a nice surprise. I'm guessing this data is from when "Die with a smile" just came out.
I already see some problems with the dataset: Songs are saved multiple times (not sure if they are different versions, or just the same versions e.g. as single and as part of an album) However, as title is saved here, it should be no problem removing duplicates.

In [50]:
display(high[:5])

,energy,tempo,danceability,playlist_genre,loudness,liveness,valence,track_artist,time_signature,speechiness,...,instrumentalness,track_album_id,mode,key,duration_ms,acousticness,id,playlist_subgenre,type,playlist_id
676,0.592,157.969,0.521,pop,-7.777,0.122,0.535,"Lady Gaga, Bruno Mars",3,0.0304,...,0.0,10FLjwfpbxLmW8c25Xyc2N,0,6,251668,0.3080,2plbrEY59IikOBgBGLjaoe,global,audio_features,37i9dQZEVXbMDoHDwVN2tF
455,0.592,157.969,0.521,gaming,-7.777,0.122,0.535,"Lady Gaga, Bruno Mars",3,0.0304,...,0.0,10FLjwfpbxLmW8c25Xyc2N,0,6,251668,0.3080,2plbrEY59IikOBgBGLjaoe,modern,audio_features,37i9dQZF1DWTyiBJ6yEqeu
0,0.592,157.969,0.521,pop,-7.777,0.122,0.535,"Lady Gaga, Bruno Mars",3,0.0304,...,0.0,10FLjwfpbxLmW8c25Xyc2N,0,6,251668,0.3080,2plbrEY59IikOBgBGLjaoe,mainstream,audio_features,37i9dQZF1DXcBWIGoYBM5M
677,0.783,149.027,0.777,pop,-4.477,0.355,0.939,"ROSÉ, Bruno Mars",4,0.2600,...,0.0,2IYQwwgxgOIn7t3iF6ufFD,0,0,169917,0.0283,5vNRhkKd0yEAg8suGBpjeY,global,audio_features,37i9dQZEVXbMDoHDwVN2tF
4,0.783,149.027,0.777,pop,-4.477,0.355,0.939,"ROSÉ, Bruno Mars",4,0.2600,...,0.0,2IYQwwgxgOIn7t3iF6ufFD,0,0,169917,0.0283,5vNRhkKd0yEAg8suGBpjeY,mainstream,audio_features,37i9dQZF1DXcBWIGoYBM5M


In [51]:
low = low.sort_values(by='track_popularity', ascending=False)
print("Most unpopular:\n", low[["track_name", "track_id", "track_artist", "track_album_name", "track_popularity"]].tail(5))

Most unpopular:
                   track_name                track_id   track_artist  \
3009          Hit The Ground  1we3IyLdDkmtZctE9xbFgm  Lilly Ahlberg   
2274      A Vaibele A Tsnien  03bEE0edmcuf9iraKHlENO    Dave Tarras   
3081                 Make It  19bfMEAOJGHEAL2pS3gNoS  Daniel Schulz   
3128  Blue Flag / Fun Things  6VvRwqQpgWU1WclXgCOKI8  Jesper Jenset   
3102            It Ain't You  4egXSi9VmYuJaq6Xa1ECQm      Hedda Mae   

                                track_album_name  track_popularity  
3009                              Hit The Ground                11  
2274  Yiddish-American Klezmer Music - 1925-1956                11  
3081                                     Make It                11  
3128                      Blue Flag / Fun Things                11  
3102                         This Might Get Loud                11  


## Cleaning and combining the 2 datasets
As I know the "Threshold" by which a song is classified as high/low popular I can just merge the two datasets. (however I still wonder how this values came to be)\
I also want to remove duplicates. Above I can see that in this case the 3 duplicates of "Die With A Smile" actually have the same track_id.
I do however wonder, if there may also be duplicates with just the same names (i.e. different versions of the song)\
I'm not quite sure yet, how I want to handle that. But first I want to test if there are any such cases anyway

1. Combining the datasets

In [52]:
print(high.columns, "\n", low.columns) # Looking manually they seem to have the same columns, but for some reason in different order
print(sorted(high.columns) == sorted(low.columns)) # Sorting them to compare their column names elementwise

Index(['energy', 'tempo', 'danceability', 'playlist_genre', 'loudness',
       'liveness', 'valence', 'track_artist', 'time_signature', 'speechiness',
       'track_popularity', 'track_href', 'uri', 'track_album_name',
       'playlist_name', 'analysis_url', 'track_id', 'track_name',
       'track_album_release_date', 'instrumentalness', 'track_album_id',
       'mode', 'key', 'duration_ms', 'acousticness', 'id', 'playlist_subgenre',
       'type', 'playlist_id'],
      dtype='object') 
 Index(['time_signature', 'track_popularity', 'speechiness', 'danceability',
       'playlist_name', 'track_artist', 'duration_ms', 'energy',
       'playlist_genre', 'playlist_subgenre', 'track_href', 'track_name',
       'mode', 'uri', 'type', 'track_album_release_date', 'analysis_url', 'id',
       'instrumentalness', 'track_album_id', 'playlist_id', 'track_id',
       'valence', 'key', 'tempo', 'loudness', 'acousticness', 'liveness',
       'track_album_name'],
      dtype='object')
True


As their equality check returned true, we can just go ahead and combine the two

In [53]:
songs = pd.concat([low, high], ignore_index=True).sort_values(by='track_popularity', ascending=False)
print(songs.head(5)["track_popularity"], songs.tail(5)["track_popularity"]) # Check if correctly concatenated & sorted

3145    100
3147    100
3146    100
3149     98
3148     98
Name: track_popularity, dtype: int64 3143    11
3129    11
3130    11
3128    11
3141    11
Name: track_popularity, dtype: int64


2. Check for duplicates

In [54]:
idDupes = songs.duplicated(subset=['track_id'], keep=False)
nameDupes = songs.duplicated(subset=['track_name', 'track_artist'], keep=False)


print(
    "Duplicates by track_id:",
    idDupes.sum()
)
print(
    "Duplicates by title and artist:",
    nameDupes.sum()
)

Duplicates by track_id: 631
Duplicates by title and artist: 671


Another problem I noticed was that the dataset doesn't save the genre of the song, but of the playlist it's taken from\
$\Rightarrow$ Removing duplicates from different playlist would remove genres of the song (i.e. missing value 'track_genre'... Why did they not include that?)\
What genre to choose? e.g. 'die with a smile' in gaming playlist with 'gaming' genre... does not really fit...
Maybe do a majority vote on all dupes for the main genre and put all other subgenre array?

In [55]:
display(
    songs[idDupes]
)

,time_signature,track_popularity,speechiness,danceability,playlist_name,track_artist,duration_ms,energy,playlist_genre,playlist_subgenre,...,track_album_id,playlist_id,track_id,valence,key,tempo,loudness,acousticness,liveness,track_album_name
3145,3.0,100,0.0304,0.521,Global Top 50,"Lady Gaga, Bruno Mars",251668.0,0.592,pop,global,...,10FLjwfpbxLmW8c25Xyc2N,37i9dQZEVXbMDoHDwVN2tF,2plbrEY59IikOBgBGLjaoe,0.535,6.0,157.969,-7.777,0.3080,0.1220,Die With A Smile
3147,3.0,100,0.0304,0.521,Today's Top Hits,"Lady Gaga, Bruno Mars",251668.0,0.592,pop,mainstream,...,10FLjwfpbxLmW8c25Xyc2N,37i9dQZF1DXcBWIGoYBM5M,2plbrEY59IikOBgBGLjaoe,0.535,6.0,157.969,-7.777,0.3080,0.1220,Die With A Smile
3146,3.0,100,0.0304,0.521,Top Gaming Tracks,"Lady Gaga, Bruno Mars",251668.0,0.592,gaming,modern,...,10FLjwfpbxLmW8c25Xyc2N,37i9dQZF1DWTyiBJ6yEqeu,2plbrEY59IikOBgBGLjaoe,0.535,6.0,157.969,-7.777,0.3080,0.1220,Die With A Smile
3149,4.0,98,0.2600,0.777,Today's Top Hits,"ROSÉ, Bruno Mars",169917.0,0.783,pop,mainstream,...,2IYQwwgxgOIn7t3iF6ufFD,37i9dQZF1DXcBWIGoYBM5M,5vNRhkKd0yEAg8suGBpjeY,0.939,0.0,149.027,-4.477,0.0283,0.3550,APT.
3148,4.0,98,0.2600,0.777,Global Top 50,"ROSÉ, Bruno Mars",169917.0,0.783,pop,global,...,2IYQwwgxgOIn7t3iF6ufFD,37i9dQZEVXbMDoHDwVN2tF,5vNRhkKd0yEAg8suGBpjeY,0.939,0.0,149.027,-4.477,0.0283,0.3550,APT.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1952,4.0,41,0.1080,0.590,Meditative Vibes,"Bnxn, taves",133564.0,0.576,ambient,meditative,...,3dctFKCiGPizmylLp0SN0a,3kEpLybbAlVUVimW5Evijb,5odFUgF0MS8DAnvlJpYdUd,0.694,1.0,81.964,-6.530,0.4650,0.0876,"Sincerely, Benson"
1943,4.0,41,0.2050,0.841,Arab X,Bnxn,163224.0,0.430,arabic,modern,...,3dctFKCiGPizmylLp0SN0a,3gZgGRvDn9tTUy8n7699pD,7f3xivnGz4HU0UigVxvlEe,0.670,7.0,99.933,-6.006,0.6980,0.1010,"Sincerely, Benson"
1933,4.0,41,0.1080,0.590,Arab X,"Bnxn, taves",133564.0,0.576,arabic,modern,...,3dctFKCiGPizmylLp0SN0a,3gZgGRvDn9tTUy8n7699pD,5odFUgF0MS8DAnvlJpYdUd,0.694,1.0,81.964,-6.530,0.4650,0.0876,"Sincerely, Benson"
2024,4.0,39,0.3850,0.752,Arab X,Olamide,127819.0,0.811,arabic,modern,...,51mowf1u3WaEYvqalsbP7M,3gZgGRvDn9tTUy8n7699pD,2ciHkdcs9fuSaMn5NhvUNA,0.563,0.0,99.699,-6.185,0.3690,0.4540,Unruly


I'm also a bit unsure of whether to remove only id duplicates or also name/artist duplicates\
I don't want duplicates to distort the distribution of the data.\
But should I also remove a vastly different version (musically speaking) of a song already in the dataset?\
I first want to take a look at the few name/artist duplicates that are no id duplicates

In [56]:
pureNameDupes = songs.index[nameDupes].difference(songs.index[idDupes])
display(
    songs.loc[pureNameDupes, ['track_name', 'track_artist', 'track_album_release_date', 'track_id', "playlist_genre", "playlist_subgenre", "track_popularity"]].sort_values(by='track_name')[:10]
)

,track_name,track_artist,track_album_release_date,track_id,playlist_genre,playlist_subgenre,track_popularity
3347,A Bar Song (Tipsy),Shaboozey,2024-05-31,5fZJQrFKWQLb7FpJXZ1g7K,pop,global,84
3185,Beautiful Things,Benson Boone,2024-04-05,3xkHsmpQCBMytMJNiDf3Ii,pop,global,89
2606,Creep,Radiohead,1993-02-22,6b2oQwSGFkzsMtQruIWm2p,rock,alternative,27
3254,Creep,Radiohead,1993-02-22,70LcF31zb1H0PyJoS1Sx1r,rock,classic,86
3315,DEGENERE 👑 (feat. benny blanco),"Myke Towers, benny blanco",2024-10-17,0qeIFEGRQqHoaJtLCl0U73,pop,mainstream,84
3584,DEGENERE 👑 (feat. benny blanco),"Myke Towers, benny blanco",2024-10-16,4xISUrw0t2elabEhKrtAJ0,latin,modern,79
4696,Edge of Seventeen - 2016 Remaster,Stevie Nicks,1981-07-27,7L3b6iaVhDVjfo52Hbvh9Z,rock,80s,69
750,Edge of Seventeen - 2016 Remaster,Stevie Nicks,2016-11-04,5LNiqEqpDc8TuqPy79kDBu,rock,classic,55
3219,Espresso,Sabrina Carpenter,2024-08-23,2HRqTpkrJO5ggZyyK6NPWz,pop,global,88
370,Favourite Girl (with Rema),"Darkoo, Rema",2024-08-08,3eRaxLLhyYHKr8oiR6NeYI,afrobeats,african,61


$\Rightarrow$ A lot of Remasters, Single versions etc. ... I should remove them too\
3. Combine track_genres:
Majority Vote of playlist_genre. The result will be the genre of the first duplicate, as this will later be the one ignored in the duplicaate removal
(some versions might have slightly different popularity values tho... fuck)

In [57]:
songs['track_genre'] = songs['playlist_genre']

dupes = songs.loc[nameDupes].sort_values(by='track_name')

genres = [songs.loc[0, 'playlist_genre']]
popularities = [songs.loc[0, 'playlist_genre']]
current = {
    'artist': songs.loc[0, 'track_artist'],
    'track_name': songs.loc[0, 'track_name'],
}

for i in songs.index:
    thisSong = {
        'artist': songs.loc[i, 'track_artist'],
        'track_name': songs.loc[i, 'track_name'],
    }

    print(current, thisSong, i)

    if (i > 0): break

# let's do this later... we have other things to do

{'artist': 'Shubh, Ikky', 'track_name': 'Baller'} {'artist': 'Lady Gaga, Bruno Mars', 'track_name': 'Die With A Smile'} 3145


lorem ipsum
### Removing duplicates

In [58]:
songs = songs.drop_duplicates(subset=['track_name', 'track_artist'], keep='first')
print( songs.duplicated(subset=['track_name', 'track_artist'], keep=False)[lambda x : x].sum() ) # Check if removal was successful

0


## Simplifying the data
I first want to drop all columns that I don't care about. I want to mainly focus on the popularity and the musical paramters of the songs.\
So I will drop any IDs, album & playlist related information aswell as the 'subgenre' column

In [61]:
cols_dropped = [
    'playlist_genre', 'track_href', 'uri', 'track_album_name',
    'playlist_name', 'analysis_url', 'track_id', 'track_album_id',
    'id', 'playlist_subgenre','type', 'playlist_id'
]

songs = songs.drop(columns=cols_dropped)

print(songs.columns, songs.shape)

Index(['time_signature', 'track_popularity', 'speechiness', 'danceability',
       'track_artist', 'duration_ms', 'energy', 'track_name', 'mode',
       'track_album_release_date', 'instrumentalness', 'valence', 'key',
       'tempo', 'loudness', 'acousticness', 'liveness', 'track_genre',
       'genre_num'],
      dtype='object') (4466, 19)


# Tests and Visualisations
Now let's have some fun! I would like to check the correlation across the columns.
I especially would like to find out, which parameters influence the popularity most.
1. Correlation Check\
I would especially find a correlation between genre and popularity interesting.
But for that I have to somehow 'convert' genre to a string\
For key this is already done actually, as they save numbers of it instead of names like "c major"

In [60]:
# display(songs[songs['track_genre'] == 'blues']) => The results might surprise you
genres = list(songs['track_genre'].unique())
songs['genre_num'] = songs['track_genre'].apply(lambda g : genres.index(g))

songs.corr(numeric_only=True)

,time_signature,track_popularity,speechiness,danceability,duration_ms,energy,mode,instrumentalness,valence,key,tempo,loudness,acousticness,liveness,genre_num
time_signature,1.000000,0.009994,0.112914,0.203429,-0.016008,0.197795,-0.012187,-0.143492,0.143400,-0.001212,-0.008815,0.219388,-0.198358,0.028861,-0.064108
track_popularity,0.009994,1.000000,0.019450,0.125332,0.015507,0.175711,0.004606,-0.242559,0.089961,0.025162,0.057477,0.196355,-0.213278,0.029435,-0.328313
speechiness,0.112914,0.019450,1.000000,0.256793,-0.091109,0.144544,-0.085727,-0.212041,0.172258,0.020413,0.073102,0.182182,-0.124993,0.097098,-0.072393
danceability,0.203429,0.125332,0.256793,1.000000,-0.133179,0.409608,-0.127975,-0.358730,0.529515,0.026373,0.028718,0.572765,-0.398303,0.005614,-0.111780
duration_ms,-0.016008,0.015507,-0.091109,-0.133179,1.000000,0.123031,0.033634,-0.142869,-0.030414,0.004489,0.035157,0.071829,-0.120379,-0.002642,0.001958
energy,0.197795,0.175711,0.144544,0.409608,0.123031,1.000000,-0.079402,-0.568047,0.503735,0.043914,0.201377,0.801844,-0.757276,0.196975,-0.313976
mode,-0.012187,0.004606,-0.085727,-0.127975,0.033634,-0.079402,1.000000,0.027516,-0.071308,-0.152557,0.004752,-0.101220,0.051331,-0.017379,-0.010972
instrumentalness,-0.143492,-0.242559,-0.212041,-0.358730,-0.142869,-0.568047,0.027516,1.000000,-0.435965,-0.027230,-0.132790,-0.639653,0.513360,-0.126918,0.432405
valence,0.143400,0.089961,0.172258,0.529515,-0.030414,0.503735,-0.071308,-0.435965,1.000000,0.032741,0.093143,0.482194,-0.367515,0.068287,-0.159483
key,-0.001212,0.025162,0.020413,0.026373,0.004489,0.043914,-0.152557,-0.027230,0.032741,1.000000,0.012803,0.047567,-0.028272,0.018733,0.009650


$\Rightarrow$ a lot of interesting stuff... Too many vales to compute visually from this table.\
But I see a lot of significant values (|cor| >= 0.5) between the song characteristics, spotify calculates (danceability etc.)\
E.g. Instrumentalness seems to harm energy (cor = -0.56) according to the data\
The most important factor for track_popularity does seem to be genre (However not sure, whether me making it numeric is a proper translation for the relation between genres...) It also might just be because I took the list from the popularity ordered dataset. So the genres should tend to be ordered by popularity...
## Boxplot for Genres
Let's simplify the data a little by taking only the 10 most popular genres\
To do this, I will calculate the mean track_popularity for each genre

In [81]:
popularities = songs.groupby('track_genre')['track_popularity'].mean().reset_index().sort_values(by='track_popularity', ascending=False)

pop_genres = popularities['track_genre'].to_numpy()[:10]

print(pop_genres)

['r&b' 'gaming' 'k-pop' 'punk' 'metal' 'rock' 'j-pop' 'hip-hop' 'indie'
 'pop']
